<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/08_01_22_SBERT%2BGAT_Embeddings_NN_Politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

notebook to concatenate the SBERT (ML) and GAT (graph) embeddings and input to a base Machine Learning model. Initial step to testing multimodality in this regard. The ML model reported good results before for RQ1, so using the same architecture.

In [ ]:
#%%capture
!pip install -U sentence-transformers wandb

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

In [ ]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

# Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [ ]:
dataset_id = 'politifact'

In [ ]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv", converters=converters)
df.head()

,Unnamed: 0,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,0,Actress Emma Stone ‘For the first time in his...,NaN,[],"[1020554564334964741, 1020817527046197248, 106...",fake,NaN,[],2911,7.976595,0,0.0
1,1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0
2,2,Friendly Fire … Charlottesville Car attacker ...,NaN,[],"[3265439004, 3250621593, 3253922920, 326691851...",fake,NaN,[],24,3.218876,0,0.0
3,3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,[],"[911971426571255810, 1036749614853103616, 1033...",fake,mightynest.com/learn/research-by-concern/dange...,[],2569,7.851661,0,0.0
4,4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...,[],0,0.000000,0,0.0


# Using the embeddings for title and text from previous experiments

In [ ]:
title_embedding_file = base_dir/f"{dataset_id}_sbert_title_embeddings.npy"

if title_embedding_file.exists():
    title_embeddings = np.load(title_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    titles = df.title.tolist()
    title_embeddings = model.encode(titles, show_progress_bar=True)
    np.save(title_embedding_file, title_embeddings)

In [ ]:
text_embedding_file = base_dir/f"{dataset_id}_sbert_fulltext_embeddings.npy"

if text_embedding_file.exists():
    text_embeddings = np.load(text_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    texts = (df.title + "\n" + df.text).tolist()
    text_embeddings = model.encode(texts, show_progress_bar=True)
    np.save(text_embedding_file, text_embeddings)

In [ ]:
graph_embeddings = np.load(base_dir/'politifact_gat_embeddings.npy')

Adding other features like number of tweets amd retweets for each article from the politifact dataset.

In [ ]:
df["num_retweets"] = df.retweets.map(lambda x: len(set(x)))
df["log_num_retweets"] = np.log1p(df.num_retweets.to_numpy())
df["num_tweets"] = df.tweets.map(lambda x: len(set([t["id"] for t in x])))
df["log_num_tweets"] = np.log1p(df.num_tweets.to_numpy())

# Training - Saving the machine learning model specifications

In [ ]:
import tensorflow as tf

In [ ]:
def get_model(d_h=64, p_in:float=0.2, p_out:float=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Dropout(p_in), # input dropout
        tf.keras.layers.Dense(d_h, activation='relu'),
        tf.keras.layers.Dropout(p_out),  
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model()
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    logits = model.predict(X_test).squeeze()
    y_pred = (logits > 0.).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

In [ ]:
params = {
    "epochs":200,
    "bs":128,
    # 'lr':1e-3
}

In [ ]:
graph_embeddings.shape

(894, 64)

In [ ]:
X = np.concatenate([
    text_embeddings, 
    df.log_num_retweets.to_numpy()[..., None], 
    df.log_num_tweets.to_numpy()[..., None],
    graph_embeddings], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 834), (894,))

In [ ]:
skf = StratifiedKFold(shuffle=True, random_state=124)

Creating a group and logging all train, test data in wandb for better tracking and visualisation.

In [ ]:
GROUP = f"{dataset_id}-fulltext-sbert+tw+gat-mlp"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="nofolds", group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/200
6/6 [==============================] - 1s 56ms/step - loss: 0.6970 - accuracy: 0.5329 - val_loss: 0.4622 - val_accuracy: 0.7486 - _timestamp: 1659435400.0000 - _runtime: 3.0000
Epoch 2/200
6/6 [==============================] - 0s 18ms/step - loss: 0.4142 - accuracy: 0.8308 - val_loss: 0.3410 - val_accuracy: 0.8883 - _timestamp: 1659435400.0000 - _runtime: 3.0000
Epoch 3/200
6/6 [==============================] - 0s 21ms/step - loss: 0.3192 - accuracy: 0.8909 - val_loss: 0.2719 - val_accuracy: 0.8939 - _timestamp: 1659435400.0000 - _runtime: 3.0000
Epoch 4/200
6/6 [==============================] - 0s 17ms/step - loss: 0.2641 - accuracy: 0.9133 - val_loss: 0.2327 - val_accuracy: 0.9162 - _timestamp: 1659435400.0000 - _runtime: 3.0000
Epoch 5/200
6/6 [==============================] - 0s 17ms/step - loss: 0.2310 - accuracy: 0.9175 - val_loss: 0.2110 - val_accuracy: 0.9162 - _timestamp: 1659435401.0000 - _runtime: 4.0000
Epoch 6/200
6/6 [==============================] - 0s 1

accuracy,▁▅▅▅▅▆▆▆▆▇▇▇▇▇▇█▇▇██▇███████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▇▇▆▆▇▆█▇▆▇▇▄▅▅▄▅▄▄▅▄▆▅██▆▆▆▇▆▇▆▆▇▆▆▆▆▇▆
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃
accuracy,0.9986
best_epoch,53


The extremely good accuracy could mean that the model may have overfit. the F1 is a better representation for the testing on a NN. 